In [1]:
import torch
import numpy as np
import random
import os
from datetime import date

from backpropamine_A2C import BP_RNetwork
from BP_A2C.BP_A2C_agent import A2C_Agent

%matplotlib inline

In [2]:
# Environment specific parameters
env_name = 'CartPole-v0'
n_runs = 5
n_evaluations = 100
max_steps = 200
num_episodes = 1000

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# A2C hyperparameters
entropy_coef = 0.03 
value_pred_coef = 0.1 
gammaR = 0.9
max_grad_norm = 4.0
batch_size = 128
print_every = 10
save_every = 50

In [4]:
# Adam hyperparameters
learning_rate = 1e-4 # For Adam optimizer
l2_coef = 0

In [5]:
training_seeds = np.load('rstdp_cartpole_stuff/seeds/training_seeds.npy')

In [6]:
# Create Results Directory
dirs = os.listdir('./BP_A2C/results/')
if not any('a2c_result' in d for d in dirs):
    result_id = 1
else:
    results = [d for d in dirs if 'a2c_result' in d]
    result_id = len(results) + 1

# Get today's date and add it to the results directory
d = date.today()
result_dir = 'BP_A2C/results/a2c_result_' + str(result_id) + "_{}_entropycoef_{}_valuepredcoef_{}_batchsize_{}_maxsteps_{}_\
maxgradnorm_{}_gammaR_{}_l2coef_{}_learningrate_{}_numepisodes_{}".format(
    str(d.year) + str(d.month) + str(d.day), entropy_coef, value_pred_coef, batch_size, max_steps, max_grad_norm, gammaR,
    l2_coef, learning_rate, num_episodes)
os.mkdir(result_dir)
print('Created Directory {} to store the results in'.format(result_dir))

Created Directory BP_A2C/results/a2c_result_8_20231013_entropycoef_0.03_valuepredcoef_0.1_batchsize_128_maxsteps_200_maxgradnorm_4.0_gammaR_0.9_l2coef_0_learningrate_0.0001_numepisodes_1000 to store the results in


In [7]:
smoothed_scores_dqn_all = []
dqn_completion_after = []

for i_run in range(1):
    print("Run # {}".format(i_run))
    seed = int(training_seeds[i_run])
    
    torch.manual_seed(seed)
    random.seed(seed)

    agent_net = BP_RNetwork(4, 64, 2, seed).to(device)
    
    optimizer = torch.optim.Adam(agent_net.parameters(), lr=1.0*learning_rate, eps=1e-4, weight_decay=l2_coef)
    agent = A2C_Agent(env_name, seed, agent_net, entropy_coef, value_pred_coef, gammaR,
                      max_grad_norm, max_steps, batch_size, num_episodes, optimizer, print_every,
                      save_every, i_run, result_dir)

    smoothed_scores, scores, best_average, best_average_after = agent.train_agent()

    # # policy_net = QNetwork(architecture, seed).to(device)
    # # target_net = QNetwork(architecture, seed).to(device)
    # policy_net = BP_RNetwork(4, 64, 2, seed).to(device)
    # target_net = BP_RNetwork(4, 64, 2, seed).to(device)

    # target_net.load_state_dict(policy_net.state_dict())

    # # optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate)
    # optimizer = optim.Adam(policy_net.parameters(), lr = learning_rate, weight_decay = l2_coef) 
    # agent = Agent(env_name, policy_net, target_net, architecture, batch_size,
    #               replay_memory_size, discount_factor, eps_start, eps_end, eps_decay,
    #               update_every, target_update_frequency, optimizer, learning_rate,
    #               num_episodes, max_steps, i_run, result_dir, seed, tau)
    
    # smoothed_scores, scores, best_average, best_average_after = agent.train_agent()

    # np.save(result_dir + '/scores_{}'.format(i_run), scores)
    # np.save(result_dir + '/smoothed_scores_DQN_{}'.format(i_run), smoothed_scores)

    # # save smoothed scores in list to plot later
    # dqn_completion_after.append(best_average_after)
    # smoothed_scores_dqn_all.append(smoothed_scores)
    # print("")

Run # 0
Batch size larger than 1 not implemented yet. Program will continue with batch size set to 1.


/home/luuk/Desktop/testing/.venv/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/home/luuk/Desktop/testing/.venv/lib/python3.8/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


ValueError: 'p' must be 1-dimensional

# WHEN EVALUATING, TAKE THE ARGMAX INSTEAD OF SAMPLING